# Import der notwendigen Bibliotheken

Zunächst werden alle für den Code notwendigen Pakete importiert.

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
import time
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision
from collections import OrderedDict

from torch.autograd import Variable
from PIL import Image
import PIL

from torch.optim import lr_scheduler
import copy

import json
import os
from os.path import exists

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Laden der Daten

Hier werden die Bilddaten mithilfe des Torchvisionpakets geladen. Der Datensatz ist in zwei Teile gegliedert: Training und Validierung. Für das Training werden Transformationen wie zufällige Skalierung, Zuschneiden und Spiegeln angewendet. Dies wird dem Netzwerk helfen, sich zu verallgemeinern und zu einer besseren Leistung führen. Es muss außerdem sichergestellt werden, dass die Eingangsdaten auf 224x224 Pixel skaliert werden, wie von dem später verwendeten Netzwerk gefordert.

Das Validierungsset wird verwendet, um die Leistung des Modells anhand von Daten zu messen, die es noch nicht gesehen hat. Dazu benötiget man keine Skalierung oder Rotationstransformationen, aber die Größe der Bilder muss richtig zugeschnitten werden.

Die von torchvision verfügbaren vortrainierten Netzwerke wurden auf den ImageNet-Datensatz geschult, wobei jeder Farbkanal separat normiert wurde. Für beide Sets müssen die Mittelwerte und Standardabweichungen der Bilder auf das normalisiert werden, was das Netzwerk erwartet. Für die Mittelwerte ist es [0.485, 0.456, 0.406] und für die Standardabweichungen[0.229, 0.224, 0.225]- berechnet aus den ImageNet-Bildern. Diese Werte verschieben jeden zu zentrierenden Farbkanal um 0 und reichen von -1 bis 1.

In [6]:
data_dir = 'C:/Users/Christian/Programming/Pytorch/Final Project/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
nThreads = 4
batch_size = 32
use_gpu = torch.cuda.is_available()

In [7]:
# Transformation der Bilder 
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        #transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Laden des Datensatzes

data_dir = 'flower_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid']}

# Definieren des dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}

class_names = image_datasets['train'].classes

# Label Mapping

Hier wird eine Zuordnung von Kategorienbezeichnung zu Kategoriename geladen. Diese befindet sich in der Datei cat_to_name.json. Es ist ein JSON-Objekt, das mit dem json-Modul eingelesen werden kann. Dies enthält eine Zuordnung von Zahlen zu den tatsächlichen Namen der Pflanzen.

In [8]:
import json

classes = json.loads('{"21": "fire lily", "3": "canterbury bells", "45": "bolero deep blue", "1": "pink primrose", "34": "mexican aster", "27": "prince of wales feathers", "7": "moon orchid", "16": "globe-flower", "25": "grape hyacinth", "26": "corn poppy", "79": "toad lily", "39": "siam tulip", "24": "red ginger", "67": "spring crocus", "35": "alpine sea holly", "32": "garden phlox", "10": "globe thistle", "6": "tiger lily", "93": "ball moss", "33": "love in the mist", "9": "monkshood", "102": "blackberry lily", "14": "spear thistle", "19": "balloon flower", "100": "blanket flower", "13": "king protea", "49": "oxeye daisy", "15": "yellow iris", "61": "cautleya spicata", "31": "carnation", "64": "silverbush", "68": "bearded iris", "63": "black-eyed susan", "69": "windflower", "62": "japanese anemone", "20": "giant white arum lily", "38": "great masterwort", "4": "sweet pea", "86": "tree mallow", "101": "trumpet creeper", "42": "daffodil", "22": "pincushion flower", "2": "hard-leaved pocket orchid", "54": "sunflower", "66": "osteospermum", "70": "tree poppy", "85": "desert-rose", "99": "bromelia", "87": "magnolia", "5": "english marigold", "92": "bee balm", "28": "stemless gentian", "97": "mallow", "57": "gaura", "40": "lenten rose", "47": "marigold", "59": "orange dahlia", "48": "buttercup", "55": "pelargonium", "36": "ruby-lipped cattleya", "91": "hippeastrum", "29": "artichoke", "71": "gazania", "90": "canna lily", "18": "peruvian lily", "98": "mexican petunia", "8": "bird of paradise", "30": "sweet william", "17": "purple coneflower", "52": "wild pansy", "84": "columbine", "12": "colts foot", "11": "snapdragon", "96": "camellia", "23": "fritillary", "50": "common dandelion", "44": "poinsettia", "53": "primula", "72": "azalea", "65": "californian poppy", "80": "anthurium", "76": "morning glory", "37": "cape flower", "56": "bishop of llandaff", "60": "pink-yellow dahlia", "82": "clematis", "58": "geranium", "75": "thorn apple", "41": "barbeton daisy", "95": "bougainvillea", "43": "sword lily", "83": "hibiscus", "78": "lotus lotus", "88": "cyclamen", "94": "foxglove", "81": "frangipani", "74": "rose", "89": "watercress", "73": "water lily", "46": "wallflower", "77": "passion flower", "51": "petunia"}')

# Aufbau und Schulung des Klassifikators

Nach der Aufbereitung des Datensatzen kann ein Klassifikator aufgebaut werden. Hier wird als Basis das vortrainierte ResNet18 Modell verwenden. Darauf hin wird eine Feed-Forward-Klassifikator codiert und Trainiert..

1. Laden des vortrainierten ResNet18 Netzwerkes.
2. Definition eines neuen, untrainierten Feed-Forward-Netzwerks als Klassifizierer unter Verwendung von der ReLU-Aktivierungsfunktion.
3. Trainieren der Klassifikatorebenen mittels Backpropagation über das vortrainierte Netzwerk.
4. Dokumentation des Verlusts und der Genauigkeit des Validierungssatzes, um die besten Hyperparameter zu ermitteln.

In [9]:
# laden des resnet-152 pre-trained network Modells
model = models.resnet18(pretrained=True)

# Festsetzen der Paramater, damit die Gewichtungen nicht mehr veändert werden (backpropagation)

for param in model.parameters():
    param.requires_grad = False

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [10]:
# Definition eines neuen, untrainierten Feed-Forward-Netzwerks als Klassifizierer unter Verwendung von ReLU-Aktivierungen und Dropout.
from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(512, 256)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(256, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))



# Austausch des vortrainierten Modellklassifikators durch den eigenen Klassifikator
model.fc = classifier

In [11]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # Jede Epoche hat eine Trainings- und Validierungsphase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Modell auf Trainingsmodus setzen
            else:
                model.eval()   # Modell auf Validiierungsmodus setzen

            running_loss = 0.0
            running_corrects = 0

            # Iteration über die Daten.
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                # Nullsetzen der Parametergradienten
                optimizer.zero_grad()

                # forward: Speicher des Verlaufs im Trainingsmodus
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward: Optimierung im Trainingsmodus
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

          # Verrechnung 
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Speichern des best-trainierten Modells
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best valid accuracy: {:4f}'.format(best_acc))

    # Laden der Gewichtungen aus dem best-trainierten Modell
    model.load_state_dict(best_model_wts)
    return model

In [12]:
#Trainieren des Modells
num_epochs = 20
if use_gpu:
    print ("Using GPU: "+ str(use_gpu))
    model = model.cuda()

# "negative log likelihood loss" als Loss-Funktion setzen, da LogSoftmax als Aktivierungsfunktion im Output-Layer festgelegt ist wurde
criterion = nn.NLLLoss()

#Optimierungsfunktion (Adam-Algorythmus) und Lernrate festlegen
optimizer = optim.Adam(model.fc.parameters(), lr=0.0001)

# Abfall der LR um den Faktor 0,1 alle 5 Epochen
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=20)

Epoch 1/20
----------
train Loss: 4.3230 Acc: 0.0885
valid Loss: 3.8636 Acc: 0.2384

Epoch 2/20
----------
train Loss: 3.6296 Acc: 0.2730
valid Loss: 3.0310 Acc: 0.3973

Epoch 3/20
----------
train Loss: 2.9556 Acc: 0.4167
valid Loss: 2.3688 Acc: 0.5428

Epoch 4/20
----------
train Loss: 2.4325 Acc: 0.5252
valid Loss: 1.8570 Acc: 0.6455

Epoch 5/20
----------
train Loss: 2.1847 Acc: 0.5784
valid Loss: 1.8011 Acc: 0.6638

Epoch 6/20
----------
train Loss: 2.1415 Acc: 0.6021
valid Loss: 1.7565 Acc: 0.6834

Epoch 7/20
----------
train Loss: 2.0977 Acc: 0.6056
valid Loss: 1.7339 Acc: 0.6736

Epoch 8/20
----------
train Loss: 2.0669 Acc: 0.6175
valid Loss: 1.7042 Acc: 0.6968

Epoch 9/20
----------
train Loss: 2.0308 Acc: 0.6226
valid Loss: 1.6605 Acc: 0.7054

Epoch 10/20
----------
train Loss: 2.0105 Acc: 0.6284
valid Loss: 1.6566 Acc: 0.6993

Epoch 11/20
----------
train Loss: 1.9962 Acc: 0.6303
valid Loss: 1.6572 Acc: 0.6956

Epoch 12/20
----------
train Loss: 2.0029 Acc: 0.6284
valid Los

# Speichern des Checkpoints

Nachdem das Netzwerk trainiert ist wird es abgespreichert, damit es später für Vorhersagen geladen werden kann. Zudem werden weitere Informationen wie die Anzahl der trainierten Epochen und der Optimierungsstatus (optimizer.state_dict) gespeichert.

In [ ]:
# Speichern des Trainierten Netzwerkes (Checkpoint setzen)
model.class_to_idx = dataloaders['train'].dataset.class_to_idx
model.epochs = num_epochs
checkpoint = {'input_size': [3, 224, 224],
                 'batch_size': dataloaders['train'].batch_size,
                  'output_size': 102,
                  'state_dict': model.state_dict(),
                  'data_transforms': data_transforms,
                  'optimizer_dict':optimizer.state_dict(),
                  'class_to_idx': model.class_to_idx,
                  'epoch': model.epochs}
torch.save(checkpoint, 'classifier270v2.pth')